#### Initial Setup

In [1]:
import os

In [2]:
%pwd

'E:\\RajaRajeshwari\\MyFolders\\Projects\\mlops_water_potability_prediction\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'E:\\RajaRajeshwari\\MyFolders\\Projects\\mlops_water_potability_prediction'

#### Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelFrontendConfig:
    static_dir: Path
    template_dir: Path
    dataset_schema : Path
    feature_scaler: Path
    model_path: Path

#### Configuration

In [6]:
from src.mlops_water_potability_prediction_project.constants import *
from src.mlops_water_potability_prediction_project.utilities.helpers import read_yaml

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

    def get_frontend_config(self) -> ModelFrontendConfig:
        config = self.config.web_app

        model_frontend_config = ModelFrontendConfig(
            static_dir=config.static_dir,
            template_dir=config.template_dir,
            dataset_schema=config.dataset_schema,
            feature_scaler=config.feature_scaler,
            model_path=config.model_path
        )

        return model_frontend_config

#### Component

In [7]:
import joblib
import json
import numpy as np
import pandas as pd
from src.mlops_water_potability_prediction_project import logger


class NotInRange(Exception):
    def __init__(self, message="Values entered are not in range"):
        self.message = message
        super().__init__(self.message)


class NotInFeatureColumn(Exception):
    def __init__(self, message="Values entered are not in feature columns"):
        self.message = message
        super().__init__(self.message)
        

class FrontendPrediction:
    def __init__(self, config: ModelFrontendConfig):
        self.config = config

    def form_response(self, dict_request):
        if self.validate_input(dict_request):
            scaled_data = self.feature_scale(dict_request)
            print(scaled_data)
            prediction_result = self.model_predict(scaled_data)
            if prediction_result == 1:
                response = "Potable"
            else:
                response = "Not Potable"
            return response

    def model_predict(self, data):
        try:
            model = joblib.load(self.config.model_path)
            prediction = [model.predict(data)][0]
            if 0 <= prediction <= 1:
                return prediction
            else:
                raise NotInRange
            logger.info("Predict data using model")
            return prediction
        except Exception as e:
            raise e
        
    def validate_input(self, data):
        def _validate_cols(col):
            schema = FrontendPrediction.get_schema(self.config.dataset_schema)
            actual_cols = schema.keys()
            if col not in actual_cols:
                raise NotInFeatureColumn
    
        def _validate_values(col, val):
            schema = FrontendPrediction.get_schema(self.config.dataset_schema)
            if not (schema[col]["min"] <= float(val) <= schema[col]["max"]):
                raise NotInRange
    
        for col, val in data.items():
            _validate_cols(col)
            _validate_values(col, val)
    
        return True

    def feature_scale(self, data):
        df = FrontendPrediction.create_dataframe(data)
        sc = joblib.load(self.config.feature_scaler)
        data = sc.transform(df) 
        return data

    @staticmethod
    def get_schema(schema_path):
        with open(schema_path, 'r') as f:
            schema = json.load(f)
        return schema

    @staticmethod
    def create_dataframe(dict):
        data = np.array([list(dict.values())])
        headers = list(dict.keys())
        df = pd.DataFrame(data, columns=headers)
        return df

#### Pipeline

In [9]:
data = {
    'ph': '2',
    'Hardness': '75',
    'Solids': '350',
    'Chloramines': '2',
    'Sulfate': '150',
    'Conductivity': '250',
    'Organic_carbon': '10',
    'Trihalomethanes': '10',
    'Turbidity': '5'
}
try:
    config = ConfigurationManager()
    frontend_config = config.get_frontend_config()
    frontend_config = FrontendPrediction(config=frontend_config)
    prediction = frontend_config.form_response(data)
    print(prediction)
except Exception as e:
    raise e

[2024-01-30 15:18:56,101]: INFO: helpers: YAML file: config\config.yaml loaded successfully]
[2024-01-30 15:18:56,103]: INFO: helpers: YAML file: params.yaml loaded successfully]
[2024-01-30 15:18:56,105]: INFO: helpers: YAML file: schema.yaml loaded successfully]
[[-3.21080684 -3.70909722 -2.50893968 -3.20331316 -4.41686841 -2.1911292
  -1.28323262 -3.49186624  1.31169752]]
Not Potable


C:\Users\rajar\AppData\Local\pypoetry\Cache\virtualenvs\mlops-water-potability-prediction-BSk4LJ4h-py3.10\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
